# 뉴스 클래스 예측

In [1]:
import tensorflow
from tensorflow.keras.datasets import reuters
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score #정확도 계산
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings(action='ignore') 

# 1. 모든 단어를 사용하기

In [2]:
#num_words는 상위 몇 번째 단어까지 사용할 것인지 
#num_words인자를 명시하지 않았을 경우 모든 데이터 사용
(x_train, y_train), (x_test, y_test) = reuters.load_data(test_split=0.2)

2121728/2110848 [==============================] - 0s 0us/step


In [3]:
print('훈련 샘플의 수: {}'.format(len(x_train)))
print('테스트 샘플의 수: {}'.format(len(x_test)))
num_classes = max(y_train) + 1
print('클래스의 수 : {}'.format(num_classes))

훈련 샘플의 수: 8982
테스트 샘플의 수: 2246
클래스의 수 : 46


In [4]:
#데이터 복원부분

word_index = reuters.get_word_index(path="reuters_word_index.json")

# +3을 한 숫자가 원래 숫자가됨. pad sos unk때문
index_to_word = { index+3 : word for word, index in word_index.items() }

# index_to_word에 0은 <pad>, 1은 <sos>, 2는 <unk>를 넣기
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

#훈련용 뉴스 데이터 복원
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)
    
x_train = decoded
print(len(x_train))

#테스트용 뉴스 데이터 복원
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded
print(len(x_test))

565248/550378 [==============================] - 0s 0us/step
8982
2246


In [5]:
#벡터화부분

#dtm 생성, dtm이 자체규칙에 따라 불필요하다고 판단하는 토큰을 제거함
dtmvector = CountVectorizer()
tfidf_transformer = TfidfTransformer()

# train에 대한 document - term maxtrix, tfidf 생성
x_train_dtm = dtmvector.fit_transform(x_train) #train 데이터를 DTM으로 변환
tfidfv = tfidf_transformer.fit_transform(x_train_dtm) #DTM을 TF-IDF 행렬로 변환

# test에 대한 document - term maxtrix, tfidf 생성
x_test_dtm = dtmvector.transform(x_test) 
tfidfv_test = tfidf_transformer.transform(x_test_dtm) 

In [6]:
#모델정의부분

nb = MultinomialNB() #나이브베이즈 분류기
cb = ComplementNB() #컴플리먼트 나이브 베이즈 분류기 - 나이브베이즈 분류기 개선
lr = LogisticRegression(C=10000, penalty='l2') #로지스틱회귀 - 이름은 회귀지만 분류모델
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False) # 서포트벡터머신(svm) 사용
tree = DecisionTreeClassifier(max_depth=10, random_state=0) #의사결정나무
forest = RandomForestClassifier(n_estimators=5, random_state=0) #랜덤포레스트
grbt = GradientBoostingClassifier(random_state=0) # 그라디언트부스팅

In [7]:
nb.fit(tfidfv, y_train)
predicted_n = nb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("나이브베이즈 정확도:", accuracy_score(y_test, predicted_n)) #예측값과 실제값 비교

cb.fit(tfidfv, y_train)
predicted_c = cb.predict(tfidfv_test)
print("컴플리먼트 나이브 베이즈 정확도:", accuracy_score(y_test, predicted_c))

lr.fit(tfidfv, y_train)
predicted_l = lr.predict(tfidfv_test)
print("로지스틱 회귀 정확도:", accuracy_score(y_test, predicted_l))

lsvc.fit(tfidfv, y_train)
predicted_ls = lsvc.predict(tfidfv_test)
print("서포트 벡터 머신 정확도:", accuracy_score(y_test, predicted_ls))

tree.fit(tfidfv, y_train)
predicted_t = tree.predict(tfidfv_test)
print("decision tree 정확도:", accuracy_score(y_test, predicted_t))

forest.fit(tfidfv, y_train)
predicted_f = forest.predict(tfidfv_test)
print("random forest 정확도:", accuracy_score(y_test, predicted_f))

grbt.fit(tfidfv, y_train)
predicted_gb = grbt.predict(tfidfv_test)
print("gradient boosting 정확도:", accuracy_score(y_test, predicted_gb))

나이브베이즈 정확도: 0.5997328584149599
컴플리먼트 나이브 베이즈 정확도: 0.7649154051647373
로지스틱 회귀 정확도: 0.813446126447017
서포트 벡터 머신 정확도: 0.7827248441674087
decision tree 정확도: 0.6211041852181657
random forest 정확도: 0.6544968833481746
gradient boosting 정확도: 0.7702582368655387


In [8]:
# 소프트 보팅 사용
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8187889581478184


1차 점수 - 모든 단어
* 나이브베이즈 정확도: 0.5997328584149599
* 컴플리먼트 나이브 베이즈 정확도: 0.7649154051647373
* 로지스틱 회귀 정확도: 0.813446126447017
* 서포트 벡터 머신 정확도: 0.7827248441674087
* decision tree 정확도: 0.6211041852181657
* random forest 정확도: 0.6544968833481746
* gradient boosting 정확도: 0.7702582368655387
* 보팅 : 0.8187889581478184

# 2. 빈도수 5000개 단어를 사용하기

In [9]:
#num_words는 상위 몇 번째 단어까지 사용할 것인지 
#num_words인자를 명시하지 않았을 경우 모든 데이터 사용
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=5000, test_split=0.2)

In [10]:
#데이터 복원부분

word_index = reuters.get_word_index(path="reuters_word_index.json")

# +3을 한 숫자가 원래 숫자가됨. pad sos unk때문
index_to_word = { index+3 : word for word, index in word_index.items() }

# index_to_word에 0은 <pad>, 1은 <sos>, 2는 <unk>를 넣기
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

#훈련용 뉴스 데이터 복원
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)
    
x_train = decoded
print(len(x_train))

#테스트용 뉴스 데이터 복원
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded
print(len(x_test))

8982
2246


In [11]:
#벡터화부분

#dtm 생성, dtm이 자체규칙에 따라 불필요하다고 판단하는 토큰을 제거함
dtmvector = CountVectorizer()
tfidf_transformer = TfidfTransformer()

# train에 대한 document - term maxtrix, tfidf 생성
x_train_dtm = dtmvector.fit_transform(x_train) #train 데이터를 DTM으로 변환
tfidfv = tfidf_transformer.fit_transform(x_train_dtm) #DTM을 TF-IDF 행렬로 변환

# test에 대한 document - term maxtrix, tfidf 생성
x_test_dtm = dtmvector.transform(x_test) 
tfidfv_test = tfidf_transformer.transform(x_test_dtm) 

In [12]:
#모델정의부분

nb = MultinomialNB() #나이브베이즈 분류기
cb = ComplementNB() #컴플리먼트 나이브 베이즈 분류기 - 나이브베이즈 분류기 개선
lr = LogisticRegression(C=10000, penalty='l2') #로지스틱회귀 - 이름은 회귀지만 분류모델
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False) # 서포트벡터머신(svm) 사용
tree = DecisionTreeClassifier(max_depth=10, random_state=0) #의사결정나무
forest = RandomForestClassifier(n_estimators=5, random_state=0) #랜덤포레스트
grbt = GradientBoostingClassifier(random_state=0) # 그라디언트부스팅

In [13]:
nb.fit(tfidfv, y_train)
predicted_n = nb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("나이브베이즈 정확도:", accuracy_score(y_test, predicted_n)) #예측값과 실제값 비교

cb.fit(tfidfv, y_train)
predicted_c = cb.predict(tfidfv_test)
print("컴플리먼트 나이브 베이즈 정확도:", accuracy_score(y_test, predicted_c))

lr.fit(tfidfv, y_train)
predicted_l = lr.predict(tfidfv_test)
print("로지스틱 회귀 정확도:", accuracy_score(y_test, predicted_l))

lsvc.fit(tfidfv, y_train)
predicted_ls = lsvc.predict(tfidfv_test)
print("서포트 벡터 머신 정확도:", accuracy_score(y_test, predicted_ls))

tree.fit(tfidfv, y_train)
predicted_t = tree.predict(tfidfv_test)
print("decision tree 정확도:", accuracy_score(y_test, predicted_t))

forest.fit(tfidfv, y_train)
predicted_f = forest.predict(tfidfv_test)
print("random forest 정확도:", accuracy_score(y_test, predicted_f))

grbt.fit(tfidfv, y_train)
predicted_gb = grbt.predict(tfidfv_test)
print("gradient boosting 정확도:", accuracy_score(y_test, predicted_gb))

나이브베이즈 정확도: 0.6731967943009796
컴플리먼트 나이브 베이즈 정확도: 0.7707034728406055
로지스틱 회귀 정확도: 0.8058771148708815
서포트 벡터 머신 정확도: 0.7658058771148709
decision tree 정확도: 0.6179875333926982
random forest 정확도: 0.701246660730187
gradient boosting 정확도: 0.767586821015138


In [14]:
# 소프트 보팅 사용
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8161175422974176


2차 점수 - 단어 5000개
* 나이브베이즈 정확도: 0.6731967943009796
* 컴플리먼트 나이브 베이즈 정확도: 0.7707034728406055
* 로지스틱 회귀 정확도: 0.8058771148708815
* 서포트 벡터 머신 정확도: 0.7658058771148709
* decision tree 정확도: 0.6179875333926982
* random forest 정확도: 0.701246660730187
* gradient boosting 정확도: 0.767586821015138
* 보팅 : 0.8161175422974176

# 3. 빈도수 15000개 단어를 사용하기(직접 넣은수치 - 보팅으로 인한 정확도가 소폭 하락했기 때문)

In [15]:
#num_words는 상위 몇 번째 단어까지 사용할 것인지 
#num_words인자를 명시하지 않았을 경우 모든 데이터 사용
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=15000, test_split=0.2)

In [16]:
#데이터 복원부분

word_index = reuters.get_word_index(path="reuters_word_index.json")

# +3을 한 숫자가 원래 숫자가됨. pad sos unk때문
index_to_word = { index+3 : word for word, index in word_index.items() }

# index_to_word에 0은 <pad>, 1은 <sos>, 2는 <unk>를 넣기
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

#훈련용 뉴스 데이터 복원
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)
    
x_train = decoded
print(len(x_train))

#테스트용 뉴스 데이터 복원
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded
print(len(x_test))

8982
2246


In [17]:
#벡터화부분

#dtm 생성, dtm이 자체규칙에 따라 불필요하다고 판단하는 토큰을 제거함
dtmvector = CountVectorizer()
tfidf_transformer = TfidfTransformer()

# train에 대한 document - term maxtrix, tfidf 생성
x_train_dtm = dtmvector.fit_transform(x_train) #train 데이터를 DTM으로 변환
tfidfv = tfidf_transformer.fit_transform(x_train_dtm) #DTM을 TF-IDF 행렬로 변환

# test에 대한 document - term maxtrix, tfidf 생성
x_test_dtm = dtmvector.transform(x_test) 
tfidfv_test = tfidf_transformer.transform(x_test_dtm) 

In [18]:
#모델정의부분

nb = MultinomialNB() #나이브베이즈 분류기
cb = ComplementNB() #컴플리먼트 나이브 베이즈 분류기 - 나이브베이즈 분류기 개선
lr = LogisticRegression(C=10000, penalty='l2') #로지스틱회귀 - 이름은 회귀지만 분류모델
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False) # 서포트벡터머신(svm) 사용
tree = DecisionTreeClassifier(max_depth=10, random_state=0) #의사결정나무
forest = RandomForestClassifier(n_estimators=5, random_state=0) #랜덤포레스트
grbt = GradientBoostingClassifier(random_state=0) # 그라디언트부스팅

In [19]:
nb.fit(tfidfv, y_train)
predicted_n = nb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("나이브베이즈 정확도:", accuracy_score(y_test, predicted_n)) #예측값과 실제값 비교

cb.fit(tfidfv, y_train)
predicted_c = cb.predict(tfidfv_test)
print("컴플리먼트 나이브 베이즈 정확도:", accuracy_score(y_test, predicted_c))

lr.fit(tfidfv, y_train)
predicted_l = lr.predict(tfidfv_test)
print("로지스틱 회귀 정확도:", accuracy_score(y_test, predicted_l))

lsvc.fit(tfidfv, y_train)
predicted_ls = lsvc.predict(tfidfv_test)
print("서포트 벡터 머신 정확도:", accuracy_score(y_test, predicted_ls))

tree.fit(tfidfv, y_train)
predicted_t = tree.predict(tfidfv_test)
print("decision tree 정확도:", accuracy_score(y_test, predicted_t))

forest.fit(tfidfv, y_train)
predicted_f = forest.predict(tfidfv_test)
print("random forest 정확도:", accuracy_score(y_test, predicted_f))

grbt.fit(tfidfv, y_train)
predicted_gb = grbt.predict(tfidfv_test)
print("gradient boosting 정확도:", accuracy_score(y_test, predicted_gb))

나이브베이즈 정확도: 0.6331255565449688
컴플리먼트 나이브 베이즈 정확도: 0.7720391807658059
로지스틱 회귀 정확도: 0.8125556544968834
서포트 벡터 머신 정확도: 0.7742653606411398
decision tree 정확도: 0.6193232413178985
random forest 정확도: 0.6714158504007124
gradient boosting 정확도: 0.7707034728406055


In [20]:
# 소프트 보팅 사용
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8165627782724845


3차 점수 - 단어 15000개
* 나이브베이즈 정확도: 0.6331255565449688
* 컴플리먼트 나이브 베이즈 정확도: 0.7720391807658059
* 로지스틱 회귀 정확도: 0.8125556544968834
* 서포트 벡터 머신 정확도: 0.7742653606411398
* decision tree 정확도: 0.6193232413178985
* random forest 정확도: 0.6714158504007124
* gradient boosting 정확도: 0.7707034728406055
* 보팅 : 0.8165627782724845

# 4. 빈도수 30000개 단어를 사용하기(직접 넣은수치 -  단어 수를 15000개로 지정하였을 때 5000개보다 조금 더 나은 보팅값 향상이 있었기 때문에 추가로 설정)

In [21]:
#num_words는 상위 몇 번째 단어까지 사용할 것인지 
#num_words인자를 명시하지 않았을 경우 모든 데이터 사용
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=30000, test_split=0.2)

In [22]:
#데이터 복원부분

word_index = reuters.get_word_index(path="reuters_word_index.json")

# +3을 한 숫자가 원래 숫자가됨. pad sos unk때문
index_to_word = { index+3 : word for word, index in word_index.items() }

# index_to_word에 0은 <pad>, 1은 <sos>, 2는 <unk>를 넣기
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

#훈련용 뉴스 데이터 복원
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)
    
x_train = decoded
print(len(x_train))

#테스트용 뉴스 데이터 복원
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded
print(len(x_test))

8982
2246


In [23]:
#벡터화부분

#dtm 생성, dtm이 자체규칙에 따라 불필요하다고 판단하는 토큰을 제거함
dtmvector = CountVectorizer()
tfidf_transformer = TfidfTransformer()

# train에 대한 document - term maxtrix, tfidf 생성
x_train_dtm = dtmvector.fit_transform(x_train) #train 데이터를 DTM으로 변환
tfidfv = tfidf_transformer.fit_transform(x_train_dtm) #DTM을 TF-IDF 행렬로 변환

# test에 대한 document - term maxtrix, tfidf 생성
x_test_dtm = dtmvector.transform(x_test) 
tfidfv_test = tfidf_transformer.transform(x_test_dtm) 

In [24]:
#모델정의부분

nb = MultinomialNB() #나이브베이즈 분류기
cb = ComplementNB() #컴플리먼트 나이브 베이즈 분류기 - 나이브베이즈 분류기 개선
lr = LogisticRegression(C=10000, penalty='l2') #로지스틱회귀 - 이름은 회귀지만 분류모델
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False) # 서포트벡터머신(svm) 사용
tree = DecisionTreeClassifier(max_depth=10, random_state=0) #의사결정나무
forest = RandomForestClassifier(n_estimators=5, random_state=0) #랜덤포레스트
grbt = GradientBoostingClassifier(random_state=0) # 그라디언트부스팅

In [25]:
nb.fit(tfidfv, y_train)
predicted_n = nb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("나이브베이즈 정확도:", accuracy_score(y_test, predicted_n)) #예측값과 실제값 비교

cb.fit(tfidfv, y_train)
predicted_c = cb.predict(tfidfv_test)
print("컴플리먼트 나이브 베이즈 정확도:", accuracy_score(y_test, predicted_c))

lr.fit(tfidfv, y_train)
predicted_l = lr.predict(tfidfv_test)
print("로지스틱 회귀 정확도:", accuracy_score(y_test, predicted_l))

lsvc.fit(tfidfv, y_train)
predicted_ls = lsvc.predict(tfidfv_test)
print("서포트 벡터 머신 정확도:", accuracy_score(y_test, predicted_ls))

tree.fit(tfidfv, y_train)
predicted_t = tree.predict(tfidfv_test)
print("decision tree 정확도:", accuracy_score(y_test, predicted_t))

forest.fit(tfidfv, y_train)
predicted_f = forest.predict(tfidfv_test)
print("random forest 정확도:", accuracy_score(y_test, predicted_f))

grbt.fit(tfidfv, y_train)
predicted_gb = grbt.predict(tfidfv_test)
print("gradient boosting 정확도:", accuracy_score(y_test, predicted_gb))

나이브베이즈 정확도: 0.5997328584149599
컴플리먼트 나이브 베이즈 정확도: 0.7653606411398041
로지스틱 회귀 정확도: 0.8103294746215495
서포트 벡터 머신 정확도: 0.7764915405164737
decision tree 정확도: 0.622439893143366
random forest 정확도: 0.6638468388245771
gradient boosting 정확도: 0.7658058771148709


In [26]:
# 소프트 보팅 사용
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8130008904719501


4차 점수 - 단어 30000개
* 나이브베이즈 정확도: 0.5997328584149599
* 컴플리먼트 나이브 베이즈 정확도: 0.7653606411398041
* 로지스틱 회귀 정확도: 0.8103294746215495
* 서포트 벡터 머신 정확도: 0.7764915405164737
* decision tree 정확도: 0.622439893143366
* random forest 정확도: 0.6638468388245771
* gradient boosting 정확도: 0.7658058771148709
* 보팅 : 0.8130008904719501

# 5. 빈도수 20000개 단어를 사용하기(직접 넣은수치 - 30000개의 단어를 사용하였을 때 보팅 값에서 성능 저하가 있었기 때문에 기존 가장 뛰어난 값을 가진 15000에서 조금만 늘린 20000개를 사용 )

In [27]:
#num_words는 상위 몇 번째 단어까지 사용할 것인지 
#num_words인자를 명시하지 않았을 경우 모든 데이터 사용
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=20000, test_split=0.2)

In [28]:
#데이터 복원부분

word_index = reuters.get_word_index(path="reuters_word_index.json")

# +3을 한 숫자가 원래 숫자가됨. pad sos unk때문
index_to_word = { index+3 : word for word, index in word_index.items() }

# index_to_word에 0은 <pad>, 1은 <sos>, 2는 <unk>를 넣기
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

#훈련용 뉴스 데이터 복원
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)
    
x_train = decoded
print(len(x_train))

#테스트용 뉴스 데이터 복원
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded
print(len(x_test))

8982
2246


In [29]:
#벡터화부분

#dtm 생성, dtm이 자체규칙에 따라 불필요하다고 판단하는 토큰을 제거함
dtmvector = CountVectorizer()
tfidf_transformer = TfidfTransformer()

# train에 대한 document - term maxtrix, tfidf 생성
x_train_dtm = dtmvector.fit_transform(x_train) #train 데이터를 DTM으로 변환
tfidfv = tfidf_transformer.fit_transform(x_train_dtm) #DTM을 TF-IDF 행렬로 변환

# test에 대한 document - term maxtrix, tfidf 생성
x_test_dtm = dtmvector.transform(x_test) 
tfidfv_test = tfidf_transformer.transform(x_test_dtm) 

In [30]:
#모델정의부분

nb = MultinomialNB() #나이브베이즈 분류기
cb = ComplementNB() #컴플리먼트 나이브 베이즈 분류기 - 나이브베이즈 분류기 개선
lr = LogisticRegression(C=10000, penalty='l2') #로지스틱회귀 - 이름은 회귀지만 분류모델
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False) # 서포트벡터머신(svm) 사용
tree = DecisionTreeClassifier(max_depth=10, random_state=0) #의사결정나무
forest = RandomForestClassifier(n_estimators=5, random_state=0) #랜덤포레스트
grbt = GradientBoostingClassifier(random_state=0) # 그라디언트부스팅

In [31]:
nb.fit(tfidfv, y_train)
predicted_n = nb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("나이브베이즈 정확도:", accuracy_score(y_test, predicted_n)) #예측값과 실제값 비교

cb.fit(tfidfv, y_train)
predicted_c = cb.predict(tfidfv_test)
print("컴플리먼트 나이브 베이즈 정확도:", accuracy_score(y_test, predicted_c))

lr.fit(tfidfv, y_train)
predicted_l = lr.predict(tfidfv_test)
print("로지스틱 회귀 정확도:", accuracy_score(y_test, predicted_l))

lsvc.fit(tfidfv, y_train)
predicted_ls = lsvc.predict(tfidfv_test)
print("서포트 벡터 머신 정확도:", accuracy_score(y_test, predicted_ls))

tree.fit(tfidfv, y_train)
predicted_t = tree.predict(tfidfv_test)
print("decision tree 정확도:", accuracy_score(y_test, predicted_t))

forest.fit(tfidfv, y_train)
predicted_f = forest.predict(tfidfv_test)
print("random forest 정확도:", accuracy_score(y_test, predicted_f))

grbt.fit(tfidfv, y_train)
predicted_gb = grbt.predict(tfidfv_test)
print("gradient boosting 정확도:", accuracy_score(y_test, predicted_gb))

나이브베이즈 정확도: 0.6193232413178985
컴플리먼트 나이브 베이즈 정확도: 0.7671415850400712
로지스틱 회귀 정확도: 0.8098842386464826
서포트 벡터 머신 정확도: 0.7804986642920748
decision tree 정확도: 0.6211041852181657
random forest 정확도: 0.6714158504007124
gradient boosting 정확도: 0.7702582368655387


In [32]:
# 소프트 보팅 사용
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8178984861976848


5차 점수 - 단어 20000개
* 나이브베이즈 정확도: 0.6193232413178985
* 컴플리먼트 나이브 베이즈 정확도: 0.7671415850400712
* 로지스틱 회귀 정확도: 0.8098842386464826
* 서포트 벡터 머신 정확도: 0.7804986642920748
* decision tree 정확도: 0.6211041852181657
* random forest 정확도: 0.6714158504007124
* gradient boosting 정확도: 0.7702582368655387
* 보팅 : 8178984861976848

모든 단어를 사용한 것을 제외하고 단어 수를 20000개로 설정하였을 때 보팅값이 가장 높게 나왔다.
너무 많은 단어를 사용하면 좋은 결과를얻지 못하는 경우가 많다고 했는데, 모든 단어를 적용하여도 단어의 수가 적은 편이기 때문에 모든 단어를 사용한 결과값이 가장 많이 나오지 않나로 생각하고 있다.
모든 단어를 사용하는 것보다 단어 20000개를 사용한 것을 바탕으로 딥러닝을 진행해보고자 한다.

# 6. LSTM을 사용하여 비교(20000개의 단어를 사용)

In [2]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=20000, test_split=0.2)
num_classes = max(y_train) + 1

2121728/2110848 [==============================] - 0s 0us/step


In [3]:
#데이터 복원부분

word_index = reuters.get_word_index(path="reuters_word_index.json")

# +3을 한 숫자가 원래 숫자가됨. pad sos unk때문
index_to_word = { index+3 : word for word, index in word_index.items() }

# index_to_word에 0은 <pad>, 1은 <sos>, 2는 <unk>를 넣기
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

#훈련용 뉴스 데이터 복원
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)
    
x_train = decoded
print(len(x_train))

#테스트용 뉴스 데이터 복원
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded
print(len(x_test))

565248/550378 [==============================] - 0s 0us/step
8982
2246


In [4]:
(rnn_x_train, rnn_y_train), (rnn_x_test, rnn_y_test) = reuters.load_data(num_words=20000, test_split=0.2)
print(len(rnn_x_train), len(rnn_x_test))
max_len = max(len(l) for l in np.concatenate((rnn_x_train, rnn_x_test), axis=0))
print('max_len : ', max_len)

8982 2246
max_len :  2376


In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

rnn_x_train = pad_sequences(rnn_x_train, maxlen=max_len)
rnn_x_test = pad_sequences(rnn_x_test, maxlen=max_len)

rnn_y_train = to_categorical(rnn_y_train, num_classes=46)
rnn_y_test = to_categorical(rnn_y_test, num_classes=46)

rnn_x_train = rnn_x_train[1000:]
rnn_y_train = rnn_y_train[1000:]
rnn_x_val = rnn_x_train[:1000]
rnn_y_val = rnn_y_train[:1000]

In [13]:
from tensorflow import keras

vocab_size = len(index_to_word)
print(vocab_size)
word_vector_dim = 120

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim))
model.add(keras.layers.LSTM(120))
model.add(keras.layers.Dense(46, activation='selu'))
# 모델 훈련

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(rnn_x_train, rnn_y_train, epochs=50, callbacks=[es], batch_size=128, validation_data=(rnn_x_val, rnn_y_val), verbose=1)

30982
Epoch 1/50
63/63 [==============================] - 12s 170ms/step - loss: 0.1003 - accuracy: 0.3816 - val_loss: 0.0880 - val_accuracy: 0.3280
Epoch 2/50
63/63 [==============================] - 11s 168ms/step - loss: 0.0816 - accuracy: 0.3607 - val_loss: 0.0789 - val_accuracy: 0.3720
Epoch 3/50
63/63 [==============================] - 11s 169ms/step - loss: 0.0731 - accuracy: 0.4732 - val_loss: 0.0724 - val_accuracy: 0.5710
Epoch 4/50
63/63 [==============================] - 10s 167ms/step - loss: 0.0696 - accuracy: 0.5915 - val_loss: 0.0667 - val_accuracy: 0.6310
Epoch 5/50
63/63 [==============================] - 10s 165ms/step - loss: 0.0725 - accuracy: 0.6136 - val_loss: 0.1014 - val_accuracy: 0.5680
Epoch 6/50
63/63 [==============================] - 11s 167ms/step - loss: 0.0839 - accuracy: 0.5991 - val_loss: 0.1102 - val_accuracy: 0.5860
Epoch 7/50
63/63 [==============================] - 10s 164ms/step - loss: 0.0878 - accuracy: 0.5623 - val_loss: 0.0839 - val_accuracy: 

In [14]:
model.evaluate(rnn_x_test, rnn_y_test)

71/71 [==============================] - 2s 34ms/step - loss: 0.1356 - accuracy: 0.6309


[0.13564540445804596, 0.6308993697166443]

# 회고

머신러닝에 비해서 딥러닝이 제대로된 학습을 하지 못하였다. 모든 단어를 사용했던 기법을 제외한 보팅 기법에서 가장 높은 성능을 보였던 20000개의 단어를 바탕으로 딥러닝을 시도 하였는데, 20000개라는 단어가 문제인 듯 싶다.비슷한 모델에 10000개의 단어를 사용한 분의 결과는 괜찮았기 때문이다. 단순히 보팅에서 가장 높은 성과를 얻은 상위 단어의 수를 사용하는 것보다 f1스코어 등등 여러가지를 고려하는 것이 필요해 보인다.